<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from collections import defaultdict 
import random
import numpy as np
import copy


In [20]:
class Graph: 
    def __init__(self,graph): 
        self.graph = graph 
        self.ROW = len(graph) 
        self.Origingraph = copy.deepcopy(graph) 
        self.TMPgraph = copy.deepcopy(graph) 
        self.iter = 0
        #初始温度，停止温度与降温系数
        self.maxiter = 10000
        self.tmp = 1e5
        self.tmp_min = 1e-3
        self.alpha = 0.98

    def bfs(self, s, t, parent):
        visited = [False]*self.ROW
        queue = [s] 
        visited[s] = True

        while queue: 
            u = queue.pop(0) 
            for ind, val in enumerate(self.graph[u]): 
                if visited[ind] == False and val > 0 : 
                    queue.append(ind) 
                    visited[ind] = True
                    parent[ind] = u 
        return True if visited[t] else False
    
    def FordFulkerson(self, source, sink): 
        parent = [-1]*self.ROW 
  
        max_flow = 0 
        
        while self.bfs(source, sink, parent): #判断增广路径 
            path_flow, s = float("Inf"), sink
            while s != source: 
                #计算增广路径的最小流量，通过最小流量计算残差网络
                path_flow = min(path_flow, self.graph[parent[s]][s]) 
                s = parent[s] 

            max_flow += path_flow 
            v = sink 
            while v != source: #计算残差网络
                u = parent[v] 
                self.graph[u][v] -= path_flow 
                self.graph[v][u] += path_flow 
                v = parent[v] 


        return max_flow 
    

graph = [[0,  6, 10, 10,  0,  0], 
         [6,  0,  0,  4,  7,  0], 
         [10, 0,  0,  1,  14, 0], 
         [10, 4,  1,  0,  0,  5], 
         [0,  7, 14,  0,  0, 21], 
         [0, 0,  0,   5, 21,  0]] 



In [21]:
%%time
g = Graph(graph) 
source = 0; sink = 5
print ("EK Algorithm")
print(g.FordFulkerson(source,sink))
print(np.array(g.graph))


EK Algorithm
23
[[ 0  0  0  3  0  0]
 [12  0  0  5  0  0]
 [20  0  0  2  3  0]
 [17  3  0  0  0  0]
 [ 0 14 25  0  0  3]
 [ 0  0  0 10 39  0]]
Wall time: 997 µs


In [22]:
def Bfs(C, F, s, t):  # C is the capacity matrix
    n = len(C)
    queue = []
    queue.append(s)
    global level
    level = n * [0]  # initialization
    level[s] = 1  
    while queue:
        k = queue.pop(0)
        for i in range(n):
            if (F[k][i] < C[k][i]) and (level[i] == 0): # not visited
                level[i] = level[k] + 1
                queue.append(i)
    return level[t] > 0

#search augmenting path by using DFS
def Dfs(C, F, k, cp):
    tmp = cp
    if k == len(C)-1:
        return cp
    for i in range(len(C)):
        if (level[i] == level[k] + 1) and (F[k][i] < C[k][i]):
            f = Dfs(C,F,i,min(tmp,C[k][i] - F[k][i]))
            F[k][i] = F[k][i] + f
            F[i][k] = F[i][k] - f
            tmp = tmp - f
    return cp - tmp

#calculate max flow
#_ = float('inf')
def MaxFlow(C,s,t):
    n = len(C)
    F = [n*[0] for i in range(n)] # F is the flow matrix
    flow = 0
    while(Bfs(C,F,s,t)):
        flow = flow + Dfs(C,F,s,100000)
    return flow

C = [[0,  6, 10, 10,  0,  0], 
    [6,  0,  0,  4,  7,  0], 
    [10, 0,  0,  1,  14, 0], 
    [10, 4,  1,  0,  0,  5], 
    [0,  7, 14,  0,  0, 21], 
    [0, 0,  0,   5, 21,  0]] 



In [19]:
%%time
source = 0  
sink = 5    
print ("Dinic's Algorithm")
# time0 = time.time()

max_flow_value = MaxFlow(C, source, sink)
# print("Dinic using time ", time.time() - time0)

print( "max_flow_value is", max_flow_value)

Dinic's Algorithm
max_flow_value is 23
Wall time: 995 µs


In [16]:
def MaxFlowReal(C, s, t):
    n = len(C) # C is the capacity matrix
    F = [[0] * n for i in range(n)]

    # the residual capacity from u to v is C[u][v] - F[u][v]
    height = [0] * n # height of node
    excess = [0] * n # flow into node minus flow from node
    seen   = [0] * n # neighbours seen since last relabel
    # node "queue"
    nodelist = [i for i in range(n) if i != s and i != t]

     #push operation
    def push(u, v):
        send = min(excess[u], C[u][v] - F[u][v])
        F[u][v] += send
        F[v][u] -= send
        excess[u] -= send
        excess[v] += send

     #relabel operation
    def relabel(u):
        # find smallest new height making a push possible,
        # if such a push is possible at all
        min_height = float('inf')
        for v in range(n):
            if C[u][v] - F[u][v] > 0:
                min_height = min(min_height, height[v])
                height[u] = min_height + 1

    def discharge(u):
        while excess[u] > 0:
            if seen[u] < n: # check next neighbour
                v = seen[u]
                if C[u][v] - F[u][v] > 0 and height[u] > height[v]:
                    push(u, v)
                else:
                    seen[u] += 1
            else: # we have checked all neighbours. must relabel
                relabel(u)
                seen[u] = 0

    height[s] = n   # longest path from source to sink is less than n long
    excess[s] = float("inf") # send as much flow as possible to neighbours of source
    for v in range(n):
         push(s, v)

    p = 0
    while p < len(nodelist):
        u = nodelist[p]
        old_height = height[u]
        discharge(u)
        if height[u] > old_height:
            nodelist.insert(0, nodelist.pop(p)) # move to front of list
            p = 0 # start from front of list
        else:
            p += 1
    return sum(F[s])


C = [[0,  6, 10, 10,  0,  0], 
    [6,  0,  0,  4,  7,  0], 
    [10, 0,  0,  1,  14, 0], 
    [10, 4,  1,  0,  0,  5], 
    [0,  7, 14,  0,  0, 21], 
    [0, 0,  0,   5, 21,  0]] 

source = 0  
sink = 5    


Push-Relabeled's Algorithm
Push-Relabeled using time  0.0
max_flow_value is 23


In [18]:
%%time
print("Push-Relabeled's Algorithm")
max_flow_value = MaxFlowReal(C, source, sink)
print( "max_flow_value is", max_flow_value)

Push-Relabeled's Algorithm
max_flow_value is 23
Wall time: 1.01 ms


In [49]:
%%time
for i in range(10000):
    graph = [[0,  6, 10, 10,  0,  0], 
             [6,  0,  0,  4,  7,  0], 
             [10, 0,  0,  1,  14, 0], 
             [10, 4,  1,  0,  0,  5], 
             [0,  7, 14,  0,  0, 21], 
             [0, 0,  0,   5, 21,  0]] 
    g = Graph(graph) 
    source = 0; sink = 5
    maxS = g.FordFulkerson(source,sink)

print(maxS)
print ("EK Algorithm")
print(np.array(g.graph))


23
EK Algorithm
[[ 0  0  0  3  0  0]
 [12  0  0  5  0  0]
 [20  0  0  2  3  0]
 [17  3  0  0  0  0]
 [ 0 14 25  0  0  3]
 [ 0  0  0 10 39  0]]
Wall time: 1.36 s


In [50]:
%%time
for i in range(10000):
    graph = [[0,  6, 10, 10,  0,  0], 
             [6,  0,  0,  4,  7,  0], 
             [10, 0,  0,  1,  14, 0], 
             [10, 4,  1,  0,  0,  5], 
             [0,  7, 14,  0,  0, 21], 
             [0, 0,  0,   5, 21,  0]] 
    source = 0 ;sink = 5    
#     print ("Dinic's Algorithm")
    max_flow_value = MaxFlow(graph, source, sink)
print ("Dinic's Algorithm")
print( "max_flow_value is", max_flow_value)

Dinic's Algorithm
max_flow_value is 23
Wall time: 780 ms


In [51]:
%%time
for i in range(10000):
    graph = [[0,  6, 10, 10,  0,  0], 
             [6,  0,  0,  4,  7,  0], 
             [10, 0,  0,  1,  14, 0], 
             [10, 4,  1,  0,  0,  5], 
             [0,  7, 14,  0,  0, 21], 
             [0, 0,  0,   5, 21,  0]] 
    source = 0 ;sink = 5    
    
    max_flow_value = MaxFlowReal(graph, source, sink)
print("Push-Relabeled's Algorithm")
print( "max_flow_value is", max_flow_value)

Push-Relabeled's Algorithm
max_flow_value is 23
Wall time: 878 ms


In [52]:
graph

[[0, 6, 10, 10, 0, 0],
 [6, 0, 0, 4, 7, 0],
 [10, 0, 0, 1, 14, 0],
 [10, 4, 1, 0, 0, 5],
 [0, 7, 14, 0, 0, 21],
 [0, 0, 0, 5, 21, 0]]